In [6]:
from config.common import cfg
from os.path import join
import polars as pl

In [7]:
behavoirs=pl.read_parquet(join(cfg['train_data_path'],'behaviors.parquet'))
behavoirs.head()

id,user_id,time,history,impressions
i64,str,datetime[μs],list[str],list[str]
1,"""U13740""",2019-11-11 09:05:58,"[""N55189"", ""N42782"", … ""N31801""]","[""N55689-1"", ""N35729-0""]"
2,"""U91836""",2019-11-12 18:11:30,"[""N31739"", ""N6072"", … ""N25785""]","[""N20678-0"", ""N39317-0"", … ""N6890-0""]"
3,"""U73700""",2019-11-14 07:01:48,"[""N10732"", ""N25792"", … ""N18870""]","[""N50014-0"", ""N23877-0"", … ""N51570-0""]"
4,"""U34670""",2019-11-11 05:28:05,"[""N45729"", ""N2203"", … ""N51891""]","[""N35729-0"", ""N33632-0"", … ""N27581-0""]"
5,"""U8125""",2019-11-12 16:11:21,"[""N10078"", ""N56514"", … ""N33740""]","[""N39985-0"", ""N36050-0"", … ""N20527-0""]"


In [8]:
print(
    behavoirs.with_columns(
        pl.col("history").list.len().alias("history_len")
    )
    .group_by("history_len")
    .count()
    .sort("history_len")
)


shape: (255, 2)
┌─────────────┬───────┐
│ history_len ┆ count │
│ ---         ┆ ---   │
│ u32         ┆ u32   │
╞═════════════╪═══════╡
│ 1           ┆ 5400  │
│ 2           ┆ 3695  │
│ 3           ┆ 5442  │
│ 4           ┆ 6308  │
│ 5           ┆ 6101  │
│ …           ┆ …     │
│ 396         ┆ 24    │
│ 409         ┆ 7     │
│ 425         ┆ 23    │
│ 436         ┆ 14    │
│ 558         ┆ 34    │
└─────────────┴───────┘


/tmp/ipykernel_10155/447516462.py:6: DeprecationWarning: `GroupBy.count` was renamed; use `GroupBy.len` instead
  .count()


In [9]:
# 检查是否有history为0的列
print(
    behavoirs
    .select(pl.col("history").list.len().min())
)

shape: (1, 1)
┌─────────┐
│ history │
│ ---     │
│ u32     │
╞═════════╡
│ 1       │
└─────────┘


In [10]:
news = pl.read_parquet(join(cfg['data_path'], 'news.parquet'))
news.head()

news_id,category,subcategory,title,abstract,url,title_entities,abstract_entities
str,str,str,str,str,str,list[struct[6]],list[struct[6]]
"""N38974""","""sports""","""football_nfl""","""David Bakhtiari was lobbying r…","""One can only imagine what was …","""https://assets.msn.com/labs/mi…","[{""David Bakhtiari"",""P"",""Q5231066"",1.0,[0],[""David Bakhtiari""]}]","[{""David Bakhtiari"",""P"",""Q5231066"",1.0,[177, 279],[""David Bakhtiari"", ""Bakhtiari""]}, {""Green Bay Press-Gazette"",""M"",""Q3116075"",1.0,[254],[""Green Bay Press-Gazette""]}, {""Green Bay Packers"",""O"",""Q213837"",0.997,[157],[""Packers""]}]"
"""N63240""","""news""","""newscrime""","""Police investigating after bod…","""GREENSBORO, N.C. Greensboro …","""https://assets.msn.com/labs/mi…","[{""Greensboro, North Carolina"",""G"",""Q49238"",1.0,[41],[""Greensboro""]}]","[{""Greensboro, North Carolina"",""G"",""Q49238"",1.0,[0],[""GREENSBORO""]}, {""Fox8"",""M"",""Q662814"",0.996,[316],[""FOX8""]}, {""FBI Criminal Investigative Division"",""O"",""Q5424571"",1.0,[255],[""Criminal Investigative Division""]}]"
"""N27956""","""news""","""newspolitics""","""Grassley: Up to whistleblower …","""Sen. Chuck Grassley (R-Iowa) s…","""https://assets.msn.com/labs/mi…","[{""Chuck Grassley"",""P"",""Q529294"",0.99,[0],[""Grassley""]}]","[{""Chuck Grassley"",""P"",""Q529294"",0.99,[5],[""Chuck Grassley""]}, {""United States congressional delegations from Iowa"",""U"",""Q1846918"",1.0,[21],[""R-Iowa""]}, {""Donald Trump"",""P"",""Q22686"",0.999,[164],[""President Trump""]}]"
"""N9227""","""news""","""newsus""","""4 cuddly canines to adopt now …","""See listings of puppies up for…","""https://assets.msn.com/labs/mi…",[],[]
"""N14184""","""news""","""newsus""","""Man cuffed for eating sandwich…","""Bay Area Rapid Transit was sla…","""https://assets.msn.com/labs/mi…",[],"[{""Bay Area Rapid Transit"",""U"",""Q610120"",1.0,[0],[""Bay Area Rapid Transit""]}]"


In [11]:
data=pl.read_parquet(join(cfg['train_data_path'],'train_df.parquet'))
data.head()

user,history,target
str,list[str],str
"""U79044""","[""N45544"", ""N477"", ""N42470""]","""N22697"""
"""U18373""","[""N22130"", ""N24406"", … ""N40509""]","""N40785"""
"""U15844""","[""N46868"", ""N12194""]","""N28992"""
"""U19498""","[""N47765"", ""N56742"", … ""N30353""]","""N18285"""
"""U91516""",[],"""N53538"""
